In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
BATCH_SIZE = 64
SEQ_LEN = 16
EPOCHS = 4000
lr = 1e-3
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EMBEDDING_DIM = 32
torch.manual_seed(69)

In [3]:
with open('data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
# create a mapping from characters to integers
CHAR_TO_INDEX = { ch:i for i,ch in enumerate(chars) }
INDEX_TO_CHAR = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [CHAR_TO_INDEX[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([INDEX_TO_CHAR[i] for i in l]) # decoder: take a list of integers, output a string

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
pct = int(0.9*len(data))
train_data = data[:pct]
val_data = data[pct:]

In [6]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - SEQ_LEN, (BATCH_SIZE,))
    x = torch.stack([data[i:i+SEQ_LEN] for i in ix])
    y = torch.stack([data[i+1:i+SEQ_LEN+1] for i in ix])
    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y

In [7]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(EMBEDDING_DIM, head_size, bias=False)
        self.query = nn.Linear(EMBEDDING_DIM, head_size, bias=False)
        self.value = nn.Linear(EMBEDDING_DIM, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(SEQ_LEN, SEQ_LEN)))

    def forward(self, x):
        B,S,C = x.shape
        k = self.key(x)   # (B,S,C)
        q = self.query(x) # (B,S,C)
        v = self.value(x) # (B,S,C)
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, S, C) @ (B, C, S) -> (B, S, S)
        wei = wei.masked_fill(self.tril[:S, :S] == 0, float('-inf')) # (B, S, S)
        wei = F.softmax(wei, dim=-1) # (B, S, S)
        out = wei @ v # (B, S, S) @ (B, S, C) -> (B, S, C)
        return out

In [8]:
class Attention(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
        self.position_embedding_table = nn.Embedding(SEQ_LEN, EMBEDDING_DIM)
        self.head = Head(EMBEDDING_DIM)
        self.fc = nn.Linear(EMBEDDING_DIM, VOCAB_SIZE)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=DEVICE)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.head(x) # apply one head of self-attention. (B,T,C)
        logits = self.fc(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -SEQ_LEN:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [9]:
model = Attention()
optimizer = optim.Adam(model.parameters(), lr=lr)
model = model.to(DEVICE)

In [10]:
model.to(DEVICE)
for epoch in range(EPOCHS):
    model.train()
    x, y = get_batch("train")
    optimizer.zero_grad()
    y_pred, loss = model(x, y)
    loss.backward()
    optimizer.step()
    model.eval()
    x, y = get_batch("val")
    y_pred, loss = model(x, y)
    print(f"Epoch {epoch+1} Loss: {loss}")

Epoch 1 Loss: 4.177602767944336
Epoch 2 Loss: 4.156603813171387
Epoch 3 Loss: 4.135813236236572
Epoch 4 Loss: 4.117031574249268
Epoch 5 Loss: 4.090582370758057
Epoch 6 Loss: 4.088229179382324
Epoch 7 Loss: 4.078524112701416
Epoch 8 Loss: 4.060380935668945
Epoch 9 Loss: 4.054286479949951
Epoch 10 Loss: 4.031461238861084
Epoch 11 Loss: 3.9965193271636963
Epoch 12 Loss: 3.981022834777832
Epoch 13 Loss: 3.9990127086639404
Epoch 14 Loss: 3.98488187789917
Epoch 15 Loss: 3.9591548442840576
Epoch 16 Loss: 3.9164819717407227
Epoch 17 Loss: 3.9281632900238037
Epoch 18 Loss: 3.897047996520996
Epoch 19 Loss: 3.882002353668213
Epoch 20 Loss: 3.8554863929748535
Epoch 21 Loss: 3.835780143737793
Epoch 22 Loss: 3.8263204097747803
Epoch 23 Loss: 3.8159801959991455
Epoch 24 Loss: 3.770123243331909
Epoch 25 Loss: 3.7983243465423584
Epoch 26 Loss: 3.7681093215942383
Epoch 27 Loss: 3.7019400596618652
Epoch 28 Loss: 3.726969003677368
Epoch 29 Loss: 3.67033052444458
Epoch 30 Loss: 3.670977830886841
Epoch 31 L

In [12]:
# context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
context = 'All:'
context = torch.tensor(encode(context), dtype=torch.long, device=DEVICE).unsqueeze(0)
context = context.to(DEVICE)
print(decode(model.generate(context, max_new_tokens=1000)[0].tolist()))

All:
Byo omin we ldfl aw ingon ben sinch your man qyurom
CHARaly nd bu ind, bolf sa feto tee ffoensout arse the ay, sto heirt,
ARI I
Whakid sok date re nve-sn
o mun pmal arinches ound nd armeilesssthat corn, a wis?
:
Opsce!
Tout formease tin,
aurso btheer nart:
Thoud daf omner, band sitr, sthas thery novean llenl d; ghn.

I nan'greatheliuf Your?
Whor tount.
Whee theans t--wla lt hish cre ave
Wie wis yous thor biy me dany estem, way,
CER:
OF nem, ust old ass wofs ssary ant tcedy, et he inene tcant traierd elt wanom st hamy burs do
Danm wil;
Thert is,
GRDUChorue ay tiivet lligche ste ave.

Hoku han
Ang bolesom, yot sucos yarue,
Whein che hene st anoplace
TUpoen sende ds
as, it re
Bow barm ally dan:
N akreray st rem:
Woto waing, be,
S:
USAl
I, beily ndevemal ban'dn my meardi?
I his whe;
Bimint cor my Agh: the;,
Tousel hes I llde shr sto, rein,
PArferow,
An homee pal, say OMunclalik Go ousi igth ymot ite kof buris bours Le haly codvech ben,
TIn kind winas,
Bul,
ath nen,
R:
TUETRIVICAO, por